In [1]:
import MyAlgUtils
import numpy as np

In [2]:
txt = """
0 3 7 5
3 0 2 3
5 5 0 7
4 2 4 0
"""

mat = MyAlgUtils.table_to_np(txt, np.int64)
print(mat)

python_idx = np.arange(0, mat.shape[0])
exam_idx_shift = 1

def python_to_exam(idx):
    return idx+exam_idx_shift

def exam_to_python(idx):
    return idx-exam_idx_shift

def find_nearest(mat, idx):
    vec = mat[idx,:].reshape(mat.shape[0])
    return np.where(vec==np.min(vec[np.nonzero(vec)]))[0]
    
def tour_to_pairs(tour):
    pairs = []
    for i in range(0, len(tour)-1):
        a = tour[i]
        b = tour[i+1]
        pair = (a, b)
        pairs.append(pair)
    return pairs

def print_tour(tour):
    tour = np.array(tour)
    pairs = tour_to_pairs(tour)
    cost = 0
    print(f"python -> {tour}")
    print(f"exam   -> {python_to_exam(tour)}")
    for pair in pairs:
        a = python_to_exam(pair[0])
        b = python_to_exam(pair[1])
        u = pair[0]
        v = pair[1]
        c = mat[pair[0],pair[1]]
        print(f"python({u} -> {v}) | exam({a} -> {b}) | cost is {c}")
        cost += c
    print(f"total cost is {cost}")
    
def compute_cost(tour):
    pairs = tour_to_pairs(tour)
    cost = 0
    for pair in pairs:
        cost += mat[pair[0],pair[1]]
    return cost

def distance(mat, a, b):
    return mat[a,b]



NameError: name 'AlgUtils' is not defined

In [ ]:
# convert tour to python indices 

tour = [1, 2, 3, 4, 1]


pairs = tour_to_pairs(tour)
print(pairs)


[(1, 2), (2, 3), (3, 4), (4, 1)]


In [ ]:
# compute the total cost of a given tour

tour = [1, 2, 3, 4, 1]
tour = exam_to_python(np.array(tour))
print_tour(tour)


python -> [0 1 2 3 0]
exam   -> [1 2 3 4 1]
python(0 -> 1) | exam(1 -> 2) | cost is 3
python(1 -> 2) | exam(2 -> 3) | cost is 2
python(2 -> 3) | exam(3 -> 4) | cost is 7
python(3 -> 0) | exam(4 -> 1) | cost is 4
total cost is 16


In [ ]:
# use nearest neighbour method to compute a tour

def nearest(mat, start, complete):
    tour = [start]
    M = np.copy(mat)
    M[:,start] = 0
    curr = start
    while np.sum(M) > 0:
        curr = find_nearest(M, curr)[0]
        tour.append(curr)
        M[:,curr] = 0
    if(complete):
        tour.append(start)
    return tour

tour = nearest(mat, start=exam_to_python(1), complete=True)
print_tour(tour)

python -> [0 1 2 3 0]
exam   -> [1 2 3 4 1]
python(0 -> 1) | exam(1 -> 2) | cost is 3
python(1 -> 2) | exam(2 -> 3) | cost is 2
python(2 -> 3) | exam(3 -> 4) | cost is 7
python(3 -> 0) | exam(4 -> 1) | cost is 4
total cost is 16


In [ ]:
# use pilot method to compute a tour

def pilot_nearest(mat, start, complete):
    
    M = np.copy(mat)

    index = {i: True for i in range(mat.shape[0])}

    tour = [start]
    index.pop(start)

    M[:,start] = 0

    keys = list(index.keys())

    while len(keys) > 0:
        
        cost = compute_cost(tour)
        print(f"current tour: {tour}, cost={cost}")
        
        print(f"looking for best point out of {keys}")

        c_min = None
        c_best = None
        for s in keys:
            t = nearest(M, start=s, complete=False)
            if(complete):
                t.append(start)
            c = compute_cost(t)
            print(f"s={s}, t={t}, c={c}")
            if(c_min is None or c < c_min):
                c_min = c
                c_best = s
        
        print(f"best is s={s}")
        
        tour.append(c_best)
        index.pop(c_best)
        M[:,c_best] = 0
        
        keys = list(index.keys())
    
    if(complete):
        tour.append(start)

    return tour

tour = pilot_nearest(mat, start=exam_to_python(1), complete=True)
print_tour(tour)

current tour: [0], cost=0
looking for best point out of [1, 2, 3]
s=1, t=[1, 2, 3, 0], c=13
s=2, t=[2, 1, 3, 0], c=12
s=3, t=[3, 1, 2, 0], c=9
best is s=3
current tour: [0, 3], cost=5
looking for best point out of [1, 2]
s=1, t=[1, 2, 0], c=7
s=2, t=[2, 1, 0], c=8
best is s=2
current tour: [0, 3, 1], cost=7
looking for best point out of [2]
s=2, t=[2, 0], c=5
best is s=2
python -> [0 3 1 2 0]
exam   -> [1 4 2 3 1]
python(0 -> 3) | exam(1 -> 4) | cost is 5
python(3 -> 1) | exam(4 -> 2) | cost is 2
python(1 -> 2) | exam(2 -> 3) | cost is 2
python(2 -> 0) | exam(3 -> 1) | cost is 5
total cost is 14


In [ ]:
def exhaustive_tours(index, start, k, complete):

    index.pop(start)
    tours = []

    def ex(k, index, t):
        k -= 1
        if(k >= 0 and len(index.keys()) > 0):
            for i in index.keys():
                ic = index.copy()
                ic.pop(i)
                tc = t.copy()
                tc.append(i)
                ex(k, ic, tc)
        else:
            tc = t.copy()
            if(complete):
                tc.append(start)
            tours.append(tc)
    
    ex(k, index, [start])

    return tours
    

def exhaustive(mat, start, k, complete):
    index = {}
    index = {i: True for i in range(mat.shape[0])}
    tours = exhaustive_tours(index, start, k, complete=complete)
    min_cost = None
    min_idx = None
    for i, tour in enumerate(tours):
        c = compute_cost(tour)
        if(min_cost is None or c < min_cost):
            min_idx = i
            min_cost = c

    return tours[min_idx]

tour = exhaustive(mat, exam_to_python(1), k=9999, complete=True)
print_tour(tour)

python -> [0 3 1 2 0]
exam   -> [1 4 2 3 1]
python(0 -> 3) | exam(1 -> 4) | cost is 5
python(3 -> 1) | exam(4 -> 2) | cost is 2
python(1 -> 2) | exam(2 -> 3) | cost is 2
python(2 -> 0) | exam(3 -> 1) | cost is 5
total cost is 14


In [ ]:
def beam_search(mat, start, B, k):
    
    P = []
    tour = exhaustive(mat, start, k, complete=False)

    print(tour)

    P = []
    return 0


cost = beam_search(mat, exam_to_python(1), B=2, k=2)
print(f"total cost is {cost}")

[0, 1, 2]
total cost is 0


In [ ]:
# https://en.wikipedia.org/wiki/3-opt

def all_segments(n: int):
    """Generate all segments combinations"""
    return ((i, j, k)
        for i in range(n)
        for j in range(i + 2, n)
        for k in range(j + 2, n + (i > 0)))

def three_opt_segment(tour, i, j, k, allow_reverse):
    """If reversing tour[i:j] would make the tour shorter, then do it."""
    # Given tour [...A-B...C-D...E-F...]
    A, B, C, D, E, F = tour[i-1], tour[i], tour[j-1], tour[j], tour[k-1], tour[k % len(tour)]
    d0 = distance(mat, A, B) + distance(mat, C, D) + distance(mat, E, F)
    d1 = distance(mat, A, C) + distance(mat, B, D) + distance(mat, E, F)
    d2 = distance(mat, A, B) + distance(mat, C, E) + distance(mat, D, F)
    d3 = distance(mat, A, D) + distance(mat, E, B) + distance(mat, C, F)
    d4 = distance(mat, F, B) + distance(mat, C, D) + distance(mat, E, A)

    if allow_reverse and d0 > d1:
        tour[i:j] = reversed(tour[i:j])
        return -d0 + d1
    elif allow_reverse and d0 > d2:
        tour[j:k] = reversed(tour[j:k])
        return -d0 + d2
    elif allow_reverse and d0 > d4:
        tour[i:k] = reversed(tour[i:k])
        return -d0 + d4
    elif d0 > d3:
        tmp = tour[j:k] + tour[i:j]
        tour[i:k] = tmp
        return -d0 + d3
    return 0

def three_opt(tour, allow_reverse):
    """Iterative improvement based on 3 exchange."""
    while True:
        delta = 0
        for (a, b, c) in all_segments(len(tour)):
            delta += three_opt_segment(tour, a, b, c, allow_reverse)
        if delta >= 0:
            break
    return tour




In [ ]:
# non-reversing three-opt

def three_opt_non_reversing(tour, i, j, k):
    tour = tour.copy()
    tmp = np.concatenate((tour[j+1:k+1], tour[i+1:j+1]))
    tour[i+1:k+1] = tmp
    return tour

tour = [1,2,3,4,5,6,7,8,1]
tour = np.array(tour)
print(tour)

tour = exam_to_python(tour)
i = exam_to_python(2) # first index of 1st edge 
j = exam_to_python(4) # first index of 2nd edge
k = exam_to_python(6) # first index of 3rd edge
tour = three_opt_non_reversing(tour, i, j, k)
tour = python_to_exam(tour)

print(tour)

[1 2 3 4 5 6 7 8 1]
[1 2 5 6 3 4 7 8 1]
